In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
sys.path.append("../")

import src.dataset_manager as dataset_manager

In [23]:
dataset_manager.DatasetManager.list_dataset_groups()

['relations', 'sst2', 'geometry_of_truth']

In [24]:
dataset_manager.DatasetManager.list_datasets_by_group()

{'geometry_of_truth': ['sp_en_trans',
  'neg_sp_en_trans',
  'cities',
  'neg_cities',
  'smaller_than',
  'larger_than',
  'common_claim_true_false',
  'companies_true_false',
  'counterfact_true_false'],
 'sst2': ['sst2'],
 'relations': ['factual/person_plays_instrument',
  'factual/person_plays_pro_sport',
  'factual/presidents_birth_year',
  'factual/star_constellation',
  'factual/person_native_language',
  'factual/person_father',
  'factual/superhero_person',
  'factual/presidents_election_year',
  'factual/person_university',
  'factual/person_band_lead_singer',
  'factual/product_by_company',
  'factual/country_largest_city',
  'factual/pokemon_evolutions',
  'factual/food_from_country',
  'factual/company_ceo',
  'factual/landmark_on_continent',
  'factual/person_plays_position_in_sport',
  'factual/country_capital_city',
  'factual/landmark_in_country',
  'factual/person_mother',
  'factual/person_occupation',
  'factual/country_language',
  'factual/country_currency',
  'fa

In [25]:
custom_dataset = dataset_manager.DatasetManager.from_dataset_group("geometry_of_truth", batch_size=2)

In [26]:
len(custom_dataset)

22637

In [27]:
for batch in custom_dataset:
    print(batch)
    break

[TransformedExample(feature='Juhayman al-Otaybi passed away at Mecca. Is this statement true?', label='Yes', context='Juhayman al-Otaybi passed away at Mecca.', query='Is this statement {}?', answer_prompt='', raw=RawExample(feature='Juhayman al-Otaybi passed away at Mecca.', label='1')), TransformedExample(feature='KAT-TUN was founded in Calgary. Is this true?', label='No', context='KAT-TUN was founded in Calgary.', query='Is this {}?', answer_prompt='', raw=RawExample(feature='KAT-TUN was founded in Calgary.', label='0'))]


In [28]:
train, test = custom_dataset.split([0.7, 0.3])

In [29]:
print(len(train), len(test))

15846 6791


In [30]:
custom_dataset = dataset_manager.DatasetManager.from_named_datasets(["cities", "neg_cities"], batch_size=2)

In [31]:
for batch in custom_dataset:
    print(batch)
    break

[TransformedExample(feature='The city of Guiyang is in China. This statement is true. Do you agree?', label='Yes', context='The city of Guiyang is in China.', query='This statement is {}. Do you agree?', answer_prompt='', raw=RawExample(feature='The city of Guiyang is in China.', label='1')), TransformedExample(feature='The city of Mudanjiang is not in China. This statement is true. Do you agree?', label='No', context='The city of Mudanjiang is not in China.', query='This statement is {}. Do you agree?', answer_prompt='', raw=RawExample(feature='The city of Mudanjiang is not in China.', label='0'))]


In [32]:
relation_dataset = dataset_manager.DatasetManager.from_dataset_group(
    "relations",
    batch_size=4
)

In [34]:
sst2_dataset = dataset_manager.DatasetManager.from_dataset_group("sst2")

In [35]:
len(sst2_dataset)

70042